<a href="https://colab.research.google.com/github/hoihui/pkgs/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
%matplotlib inline
import matplotlib.pyplot as plt

# Logistic Regression on imdb for sentiment

https://towardsdatascience.com/sentiment-analysis-with-python-part-1-5ce197074184

In [ ]:
import urllib.request,tarfile
import os,tempfile
os.chdir(tempfile.gettempdir())
urllib.request.urlretrieve('http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz','aclImdb_v1.tar.gz')
tarfile.open('aclImdb_v1.tar.gz','r:gz').extractall()
os.chdir('aclImdb')
cmd='for split in train test; do for sentiment in pos neg; do for file in $split/$sentiment/*; do cat $file >> full_${split}.txt; echo >> full_${split}.txt; done; done; done;'
os.sysmtem(cmd)

In [24]:
import nltk, string
wnl = nltk.WordNetLemmatizer()
reviews_train=[' '.join(wnl.lemmatize(w) for w in nltk.word_tokenize(l) if w.isalpha())
               for l in open('full_train.txt')]
reviews_test=[' '.join(wnl.lemmatize(w) for w in nltk.word_tokenize(l) if w.isalpha())
              for l in open('full_test.txt')]

## BOW - 1-gram

features = 1/0 * vocab size (exist/not exist)

target = 1/0 (positive/negative)


In [32]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train)
X = cv.transform(reviews_train)
y = [1 if i < 12500 else 0 for i in range(25000)]

X_test = cv.transform(reviews_test)

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np

for c in [0.01, 0.03, 0.1, 0.3, 1]:  
    print("Accuracy for C={}: {}".format(c,
                                         np.mean(cross_val_score(LogisticRegression(C=c,solver='liblinear'), X,y, cv=5, scoring='accuracy'))
                                        )
         )

Accuracy for C=0.01: 0.8612399999999999
Accuracy for C=0.03: 0.86492
Accuracy for C=0.1: 0.8626000000000001
Accuracy for C=0.3: 0.85856
Accuracy for C=1: 0.8500400000000001


In [53]:
from sklearn.metrics import accuracy_score
model=LogisticRegression(C=0.03).fit(X,y)
print ("Final Accuracy: %s" % accuracy_score(target, model.predict(X_test)))
feature_to_coef = {
    word: coef for word, coef in zip( cv.get_feature_names(), model.coef_[0] )
}
for best_positive in sorted( feature_to_coef.items(),  key=lambda x: x[1],  reverse=True)[:5]:
    print (best_positive)    
for best_negative in sorted( feature_to_coef.items(),  key=lambda x: x[1])[:5]:
    print (best_negative)

Final Accuracy: 0.87772
('excellent', 0.8593489932761398)
('perfect', 0.6801029769414313)
('great', 0.6545590135908305)
('amazing', 0.5924591149829764)
('favorite', 0.5651963748851384)
('worst', -1.2600324008053674)
('waste', -1.0270444543800354)
('awful', -0.8978979413808582)
('boring', -0.7401303362319944)
('poorly', -0.717995970049884)


## 2-gram?

# Word2Vec

https://github.com/udacity/deep-learning-v2-pytorch/tree/master/word2vec-embeddings

## Data

In [4]:
url='https://s3.amazonaws.com/video.udacity-data.com/topher/2018/October/5bbe6499_text8/text8.zip'
import urllib.request,zipfile
import os,tempfile
os.chdir(tempfile.gettempdir())
urllib.request.urlretrieve(url,'text.zip')
zipfile.ZipFile('text.zip').extractall()
with open('text8') as f:
    text = f.read()
print(text[:100])

 anarchism originated as a term of abuse first used against early working class radicals including t


### Preprocess

In [5]:
# Tokenize
import nltk
from collections import Counter
from string import punctuation
for p in punctuation:
    text=text.replace(p,f' {p} ')

words = text.split()
cnt = Counter(words)
words=[w for w in words if cnt[w]>5]
print(words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


In [6]:
# Lookup table with descending frequency
cnt = Counter(words)
sorted_vocab = sorted(cnt, key=cnt.get, reverse=True)
int2vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
vocab2int = {word: ii for ii, word in int2vocab.items()}
integers = [vocab2int[w] for w in words]
Nw = len(words)
Nv = len(cnt)
print(f"{Nw} words, {Nv} vocabs")

16680599 words, 63641 vocabs


**Mikolov subsampling**: Discard $i^{\rm th}$ word with frequency $$P_\text{discard}(w_i)=1-\sqrt{\frac{\rm thres}{\text{total number of $w_i$ / total  number words}}},$$
So that frequent words `the`, `of`, etc. are downsampled.

In [7]:
# Mikolov subsampling
import random, numpy as np
thres = 1e-5
cnt =  Counter(integers)
P_dis = {i:1-np.sqrt(thres/(n/Nw)) for i,n in cnt.items()}
train = [i for i in integers if random.random()>P_dis[i]]
print(train[:10])

[5233, 3133, 10571, 27349, 15067, 58112, 10712, 1324, 454, 2731]


### Batch Loader

Advantage of random window size: closer words are included more frequently than distant words => higher importance to close words

In [8]:
import numpy as np
def get_surround(li, idx, minsize=1, maxsize=5): # list of neighboring elements randomly of size 2*(minsize,maxsize)
    R = np.random.randint(minsize, maxsize+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    win = li[start:idx] + li[idx+1:stop+1]    
    return list(win)
get_surround(list(range(10)), idx=5, minsize=3, maxsize=3)

[2, 3, 4, 6, 7, 8]

In [9]:
def get_batches(words, batch_size, maxsize=5): #generates (list of centre, list of surround), with duplicated element in c
    n_batches = len(words)//batch_size
    words = words[:n_batches*batch_size]    
    for idx in range(0, len(words), batch_size):
        c, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_c = batch[ii]
            batch_y = get_surround(batch, ii, maxsize)
            y.extend(batch_y)
            c.extend([batch_c]*len(batch_y))
        yield c, y
        
next(get_batches(list(range(10)), batch_size=4))

([0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3], [1, 2, 3, 0, 2, 3, 0, 1, 3, 0, 1, 2])

## SkipGram using Torch

In [10]:
import torch
cuda=torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'
torch.__version__, cuda

('1.1.0', False)

In [11]:
embedding_dim=300
print_every = 500
steps = 0
epochs = 5

### Model

SkipGram: predict surrounding words from centre word
* `nn.Embedding`: converts word to vec
* `nn.Linear`: dense embed_dim $\rightarrow$ n_vocab (the prediction)
* `nn.LogSoftmax`: converts to probability

In [12]:
from torch import nn
import torch.optim as optim

class SkipGram(nn.Module):
    def __init__(self, n_vocab, n_embed):
        super().__init__()        
        self.embed = nn.Embedding(n_vocab, n_embed)
        self.output = nn.Linear(n_embed, n_vocab)
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        x = self.embed(x)
        scores = self.output(x)
        log_ps = self.log_softmax(scores)        
        return log_ps
    
model = SkipGram(Nv, embedding_dim).to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

### Validation function

During training, want some words examples that the model considers "similar"

In [13]:
def similar_examples(embedding, sample_size=10, topk=5, device='cpu'):
    """ Returns list of exampleidx, list of list of idx similar to the corresponding exampleidx """
        
    embed_vectors = embedding.weight  # shape (Nv, embedding_dim)
    magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0) # = |b|
    
    # pick N vocab id from ranges (0,100) and (1000,1100)  (corr. frequent & infrequent words)
    valid_examples = np.array(random.sample(range(100), sample_size//2))
    valid_examples = np.append(valid_examples,
                               random.sample(range(1000,1100), sample_size//2))
    valid_examples = torch.LongTensor(valid_examples).to(device)
    
    valid_vectors = embedding(valid_examples) # shape (sample_size, embedding_dim)
    similarities = torch.mm(valid_vectors, embed_vectors.t())/magnitudes # sim=(a.b)/|a||b|  (common denom |a| discarded)
    
    _, closest_idxs = similarities.topk(topk+1) # highest similarities (includes itself)
    
    return valid_examples.to('cpu'), closest_idxs[:,1:].to('cpu') #exclude itself

### Train

In [ ]:
step = 0
for _ in range(epochs):
    
    for centre, surround in get_batches(train, 256):
        step+=1
        inputs  = torch.LongTensor(centre).to(device)
        targets = torch.LongTensor(surround).to(device)
        
        log_ps = model(inputs)
        loss = criterion(log_ps, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step%print_every: continue
        # [OPTIONAL] getting examples and similarities
        from IPython.display import clear_output
        clear_output()
        for exampleidx, closeidxs in zip(*similar_examples(model.embed, device=device)):
            print(int2vocab[exampleidx.item()] + " ~ " + ', '.join(int2vocab[idx.item()] for idx in closeidxs))

## SkipGram + Negative Sampling

https://github.com/udacity/deep-learning-v2-pytorch/blob/master/word2vec-embeddings/Negative_Sampling_Solution.ipynb

## Visualization

In [ ]:
from sklearn.manifold import TSNE
embeddings = model.embed.weight.to('cpu').data.numpy()
viz_words = 60
tsne = TSNE()
embed_tsne = tsne.fit_transform(embeddings[:viz_words, :])
fig, ax = plt.subplots(figsize=(16, 16))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int2vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)

# Classification by CNN

compared with n-gram (counting, then computing MLE).

Now representing words by vec, use CNN along time for look for pattern (larger size = longer-grams).

https://www.kaggle.com/babakgohardani/spam-detection-with-deep-learning-methods

# Language modelling by MLP